In [ ]:
import sys
import numpy as np
import pickle
from scipy.optimize import fsolve, bisect

powerfactory_path = r'C:\Program Files\DIgSILENT\PowerFactory 2020 SP4\Python\3.8'
if powerfactory_path not in sys.path:
    sys.path.append(powerfactory_path)
import powerfactory as pf

try:
    from pfcommon import *
except:
    sys.path.append('..')
    from pfcommon import *

In [ ]:
app = pf.GetApplication()
if app is None:
    raise Exception('Cannot get PowerFactory application')
else:
    print('Successfully obtained PowerFactory application.')

In [ ]:
project_name = '\\Terna_Inerzia\\IEEE39 w/ Stoch. Load & Compensator'
# project_name = '\\Terna_Inerzia\\IEEE39 w/ Stoch. Load'
err = app.ActivateProject(project_name)
if err:
    raise Exception(f'Cannot activate project {project_name}')
print(f'Successfully activated project {project_name}.')

In [ ]:
project = app.GetActiveProject()
if project is None:
    raise Exception('Cannot get active project')
print('Successfully obtained active project.')

In [ ]:
project_folders = {}
for folder_name in ('study',):
    project_folders[folder_name] = app.GetProjectFolder(folder_name)
    if project_folders[folder_name] is None:
        raise Exception(f'No folder "{folder_name}" present')
    print(f'Successfully obtained folder "{folder_name}".')

In [ ]:
generators = app.GetCalcRelevantObjects('*.ElmSym')
lines = app.GetCalcRelevantObjects('*.ElmLne')
buses = app.GetCalcRelevantObjects('*.ElmTerm')
loads = app.GetCalcRelevantObjects('*.ElmLod')
n_generators, n_lines, n_buses, n_loads = len(generators), len(lines), len(buses), len(loads)
print(f'There are {n_generators} generators.')
print(f'There are {n_lines} lines.')
print(f'There are {n_buses} buses.')
print(f'There are {n_loads} loads.')

In [ ]:
P_gen = {}
S_gen = {}
H_gen = {}

default_H = {
    'G 01': 5.0,
    'G 02': 4.33,
    'G 03': 4.47,
    'G 04': 3.57,
    'G 05': 4.33,
    'G 06': 4.35,
    'G 07': 3.77,
    'G 08': 3.47,
    'G 09': 3.45,
    'G 10': 4.20,
    'G 11': 0.1
}
coeff = 1
H = {k: coeff*v for k,v in default_H.items()}
for generator in generators:
    name = generator.loc_name
    generator_type = generator.typ_id
    generator_type.h = H[name]
    P_gen[name] = generator.pgini
    S_gen[name] = generator_type.sgn
    H_gen[name] = generator_type.h
    print(f'{name}: P = {P_gen[name]:4.0f} MW, S = {S_gen[name]:5.0f} MVA, inertia = {H_gen[name]:5.2f} s.')

In [ ]:
areas_map = {
    1: ['G 02', 'G 03', 'G 10'],
    2: ['G 04', 'G 05', 'G 06', 'G 07'],
    3: ['G 08', 'G 09'],
    4: ['G 01']
}
if n_generators == 1:
    areas_map[1].append('G 11')
H_area = {}   # inertia
E_area = {}   # energy
M_area = {}   # momentum
for area_id,generator_names in areas_map.items():
    num, den = 0,0
    for generator_name in generator_names:
        num += S_gen[generator_name] * H_gen[generator_name]
        den += S_gen[generator_name]
    H_area[area_id] = num / den 
    E_area[area_id] = num * 1e-3
    M_area[area_id] = 2 * num * 1e-3 / 60
print('Area inertias:  [{}] s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', H_area.values())))))
print('Area energies:  [{}] GW s.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', E_area.values())))))
print('Area momentums: [{}] GW s^2.'.format(', '.join(list(map(lambda s: f'{s:5.2f}', M_area.values())))))

In [ ]:
study_case_name = '1. Power Flow'

In [ ]:
if n_generators == 11:
    def cost(usetp, generator_name, app, project_folders, generators, loads, buses, study_case_name):
        idx = [gen.loc_name for gen in generators].index(generator_name)
        generators[idx].usetp = usetp
        lf = run_load_flow(app, project_folders['study'], generators, loads, buses, study_case_name)
        return lf['generators'][generator_name]['Q']
    
    generator_name = 'G 11'
    optim_usetp = bisect(cost, 0.9, 1.1, args=(generator_name, app, project_folders, generators, loads, buses, study_case_name))
    print(f'Optimal value of voltage set point for generator "{generator_name}": {optim_usetp:.4f} p.u.')

In [ ]:
lf_res = run_load_flow(app, project_folders['study'], generators, loads, buses, study_case_name)

In [ ]:
filename = 'IEEE39_LF'
if n_generators == 10:
    filename += '_with_compensator'
pickle.dump(lf_res, open(filename + '.pkl','wb'))

In [ ]:
print_load_flow(lf_res)